In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from datetime import timedelta, date
import sys

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [3]:
base_link = "https://thegreyhoundrecorder.com.au/"
add_link = "/results/bulli/2033"
link = base_link+add_link


In [4]:
raw_html = simple_get(link) #Connection complete
    
html = BeautifulSoup(raw_html, 'html.parser')

In [9]:
html_file_name='race.html'
orig_stdout = sys.stdout
f = open(html_file_name, 'w')
sys.stdout = f

print(html.prettify())

sys.stdout = orig_stdout
f.close()

In [7]:
print(type(html))

<class 'bs4.BeautifulSoup'>


In [10]:
raceSummary = html.findAll("div", {"class": "dogContainer"})
no_of_races = len(raceSummary)

In [40]:
print(raceSummary[0].prettify())

<div class="dogContainer" id="race-1">
 <table class="runnerSummary">
  <tbody>
   <tr>
    <td class="font14">
     <strong>
      R1
     </strong>
     SOUTHERN ENGINE RECONDITIONING M                      (400m)
    </td>
    <td>
     <i aria-hidden="true" class="formButton fa fa-angle-down marginLeft10 fa-lg fa-lg">
     </i>
    </td>
   </tr>
  </tbody>
 </table>
 <div class="runnerForm">
  <div class="runnerFormHeader">
   <div>
    Place
   </div>
   <div>
    Maiden - $830 - $240 -
                      $115
   </div>
   <div>
    Splits: 6.47
   </div>
  </div>
  <!-- <table class="table raceResultsTable">
                    <tbody> -->
  <!-- <tr class="runner">
													<td>1<span class="box three">3</span><span><strong><a href="/greyhounds/zambra-icon">ZAMBRA ICON</a></strong> (3)</span><br /><span class="trainerName"><strong>T:</strong>&nbsp;Marien Sullivan</span></td>
													<td>$2.70<br />&nbsp;</td>
													<td>0.00<br />&nbsp;</td>
													<td>2

In [88]:
i=0
runnerSummary = raceSummary[i].findAll("table", {"class": "runnerSummary"})

In [89]:
print(runnerSummary[0].find('td').decode_contents()[(len(runnerSummary[0].find('td').find('strong').decode_contents())+18):])

SOUTHERN ENGINE RECONDITIONING M                      (400m)
                    


In [93]:
print(raceSummary[i].findAll("div", {"class": "runnerFormHeader"})[0].findAll('div')[1].decode_contents())


Maiden - $830 - $240 -
                      $115


In [91]:
print(raceSummary[i].findAll("div", {"class": "runnerFormHeader"})[0].findAll('div')[2].decode_contents())

Splits: 6.47
